In [ ]:
%matplotlib inline

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

In [ ]:
# Hide warning messages in notebook
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# File to Load 
mouse_data_path= os.path.join("data","mouse_drug_data.csv")
clinical_data_path = os.path.join("data","clinicaltrial_data.csv")

# Read the Mouse and Drug Data and the Clinical Trial Data
mouse_drug_df = pd.read_csv(mouse_data_path)
clinical_trial_df = pd.read_csv(clinical_data_path)

# Combine the data into a single dataset
combine_data_df = pd.merge(clinical_trial_df,mouse_drug_df,on="Mouse ID",how="left")

# Display the data table for preview
combine_data_df.head()

In [ ]:
#######################Tumor Response to Treatment#######################

# Store the Mean Tumor Volume Data Grouped by Drug and Timepoint 
grouped_df = combine_data_df.groupby(["Drug","Timepoint"])
avg_tumor_df = grouped_df["Tumor Volume (mm3)"].mean()

# Convert to DataFrame
tumor_response_to_treatment = avg_tumor_df.reset_index()

# Preview DataFrame
tumor_response_to_treatment.head() 


In [ ]:
# Store the Standard Error of Tumor Volumes Grouped by Drug and Timepoint
standard_error_grouped = combine_data_df.groupby(["Drug","Timepoint"])["Tumor Volume (mm3)"]
tumor_standard_error = standard_error_grouped.sem()

# Convert to DataFrame
tumor_standard_error_df = tumor_standard_error.reset_index()

# Preview DataFrame
tumor_standard_error_df.head()

In [ ]:
# Minor Data Munging to Re-Format the Data Frames
reformat_data_tumor = tumor_response_to_treatment.pivot_table(index="Timepoint",columns="Drug", values="Tumor Volume (mm3)")

# Preview that Reformatting worked
reformat_data_tumor.head()

In [ ]:
# Generate the Plot (with Error Bars)
    
drug_list = [("Capomulin","o","red"),("Infubinol","^","blue"),("Ketapril","s","green"),("Placebo","d","black")]

for drug,marker,colors in drug_list:
    standard_error = tumor_standard_error[drug]
    tumor_response_plot = plt.errorbar(x=reformat_data_tumor.index, y=reformat_data_tumor[drug],yerr=standard_error,
                                       marker=marker, ls="--", color=colors)
    
#Show the Figure
plt.legend(loc="best")
plt.title("Tumor Response to Treatment")
plt.xlabel("Time(Days)")
plt.ylabel("Tumor Volume (mm3)")
plt.grid()


# Save the Figure
plt.savefig(os.path.join("images","tumor_response_to_treatment.png"))
plt.show()

In [ ]:
#######################Metastatic Response to Treatment#######################

# Store the Mean Met. Site Data Grouped by Drug and Timepoint
grouped_met = combine_data_df.groupby(["Drug","Timepoint"])
avg_met = grouped_met["Metastatic Sites"].mean()

# Convert to DataFrame
met_response_to_treatment = avg_met.reset_index()

# Preview DataFrame
met_response_to_treatment.head()

In [ ]:
# Store the Standard Error associated with Met. Sites Grouped by Drug and Timepoint 
met_standard_error_grouped = combine_data_df.groupby(["Drug","Timepoint"])["Metastatic Sites"]
met_standard_error = met_standard_error_grouped.sem()


# Convert to DataFrame
met_standard_error_df = met_standard_error.reset_index()

# Preview DataFrame
met_standard_error_df.head()

In [ ]:
# Minor Data Munging to Re-Format the Data Frames
reformat_data_met = met_response_to_treatment.pivot_table(index="Timepoint",columns="Drug", values="Metastatic Sites")

# Preview that Reformatting worked
reformat_data_met.head()

In [ ]:
# Generate the Plot (with Error Bars)

for drug,marker,colors in drug_list:
    standard_error = met_standard_error[drug]
    metastatic_spread_plot = plt.errorbar(x=reformat_data_met.index, y=reformat_data_met[drug], yerr=standard_error, 
                                          marker=marker, ls="--", color=colors)
    
#Show the Figure
plt.legend(loc="best")
plt.title("Metastatic Spread During Treatment")
plt.xlabel("Treatment Duration(Days)")
plt.ylabel("Met.Sites")
plt.grid()


# Save the Figure
plt.savefig(os.path.join("images","metastatic_spread_during_treatment.png"))
plt.show()

In [ ]:
#######################Survival Rates#######################

# Store the Count of Mice Grouped by Drug and Timepoint (W can pass any metric)
mouse_grouped = combine_data_df.groupby(["Drug","Timepoint"])["Mouse ID"]
mouse_count = mouse_grouped.nunique()


# Convert to DataFrame
mouse_count_df = mouse_count.reset_index()

#Rename the mouse id column
mouse_count_df = mouse_count_df.rename(columns={"Mouse ID":"Mouse Count"})

# Preview DataFrame
mouse_count_df.head()

In [ ]:
# Minor Data Munging to Re-Format the Data Frames
reformat_data_mouse = mouse_count_df.pivot_table(index="Timepoint",columns="Drug", values="Mouse Count")

# Preview the Data Frame
reformat_data_mouse.head()

In [ ]:
# Generate the Plot (Accounting for percentages)

for drug,marker,colors in drug_list:
    mouse_total = reformat_data_mouse[drug][0]
    survival_rate = (reformat_data_mouse[drug]/mouse_total)*100
    survival_treatment_plot = plt.errorbar(x=reformat_data_mouse.index, y=survival_rate, 
                                           marker=marker, ls="--", color=colors)
    
# Show the Figure
plt.legend(loc="best")
plt.title("Survival During Treatment")
plt.xlabel("Time(Days)")
plt.ylabel("Survival Rate(%)")
plt.grid()

# Save the Figure
plt.savefig(os.path.join("images","survival_during_treatment.png"))
plt.show()

In [ ]:
#######################Summary Bar Graph#######################

# Calculate the percent changes for each drug
change_value = reformat_data_tumor.iloc[-1] - reformat_data_tumor.iloc[0]

percent_change = (change_value /(reformat_data_tumor.iloc[0]))*100

# Display the data to confirm
percent_change

In [ ]:
# Orient widths. Add labels, tick marks, etc. 

only_drug_list = ["Capomulin","Infubinol","Ketapril","Placebo"]
percent_change_list = [percent_change[drug] for drug in only_drug_list]
result = percent_change > 0
percent_change_plot = plt.bar(x=only_drug_list, y=percent_change_list,
                              color = result.map({True:"red", False:"green"}))

plt.title("Tumor Change Over 45 Day Treatment")
plt.ylabel("% Tumor Volume Change")
plt.grid()


